In [17]:
#from google.colab import drive
#drive.mount('/content/drive')

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
#from osgeo import gdal

%cd windpath
%cd 12_sample


[WinError 2] 지정된 파일을 찾을 수 없습니다: 'windpath'
D:\dvc\windpath\12_sample
[WinError 2] 지정된 파일을 찾을 수 없습니다: '12_sample'
D:\dvc\windpath\12_sample
(1200, 1500, 2)


In [62]:
from tensorflow.keras.layers import Input, Conv2D, ConvLSTM2D, Dense, Flatten, Concatenate,Reshape, BatchNormalization, UpSampling2D, Conv2DTranspose
from tensorflow.keras.models import Model

time_steps = 9
width = 1500
height = 1200
channels = 2

# Terrain, landuse 이미지 데이터 입력
image_input = Input(shape=(height,width, channels), name ='image_input')

# CNN
cnn_branch = Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(image_input)
cnn_branch = Flatten()(cnn_branch)
cnn_branch = Dense(1, activation='relu')(cnn_branch)

# uz,vz 시계열 이미지 입력 레이어
lstm_input = Input(shape=(time_steps,height,width, channels), name='lstm_input')

# LSTM
lstm_branch = ConvLSTM2D(filters=32, kernel_size=(3, 3), padding='same', return_sequences=True)(lstm_input)
lstm_branch = Flatten()(lstm_branch)
lstm_branch = Dense(1, activation='relu')(lstm_branch)

# 지역풍 풍속, 높이, 풍향 정수 데이터 입력 레이어
int_input = Input(shape=(3,), name='int_input')

# 정수 분기
mlp_branch = Dense(1, activation='relu')(int_input)

# 결합
merged_branch = Concatenate()([lstm_branch,cnn_branch, mlp_branch])

# conv2dTranspose
mid_layer = Dense(225000, activation='linear', name='output_layer')(merged_branch)
mid_layer = Reshape((300, 375, 2))(mid_layer)
mid_layer = Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(mid_layer)
output_layer = Conv2DTranspose(2, (3, 3), strides=(2, 2), padding='same')(mid_layer)

# 모델 생성
inputs = [image_input,lstm_input, int_input]
model = Model(inputs=inputs, outputs=output_layer)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lstm_input (InputLayer)        [(None, 9, 1200, 15  0           []                               
                                00, 2)]                                                           
                                                                                                  
 image_input (InputLayer)       [(None, 1200, 1500,  0           []                               
                                 2)]                                                              
                                                                                                  
 conv_lstm2d_5 (ConvLSTM2D)     (None, 9, 1200, 150  39296       ['lstm_input[0][0]']             
                                0, 32)                                                      

In [55]:
from tensorflow.keras.callbacks import EarlyStopping


X1 = [] # terrain, landuse 이미지
X2 = []   # uz,vz 시계열이미지
X3 = []    # 지역풍 높이, 풍속 ,풍향 입력 데이터
Y = []   # Output data uz,vz 시계열 이미지
path_prefix = "klam21_"
for i in range(1, 2):
  for h in range(1, 3):
    for s in range(0,6):
      for d in range(0, 360, 60):
        path = path_prefix + "0000" +"{:02d}".format(i) +"_R010_H0"+ str(h) +".0_S00"+str(s)+".00_D"+"{:003d}".format(d)
        landuse = np.loadtxt(path +'/landuse.txt', delimiter = ' ', skiprows = 6, dtype = 'int')
        terrain = np.loadtxt(path +'/terrain.txt', delimiter = ' ', skiprows = 6, dtype = 'int')
        stacked_uv = np.dstack((uz,vz))
        X1.append(stacked_uv)
        X3.append((h,s,d))
        time_images = []
        for j in range(360, 3960, 360):
            uz = np.loadtxt(path+'/result/'+"0000" +"{:02d}".format(i) +"_R010_H0"+ str(h) +".0_S00"+str(s)+".00_D"+"{:003d}".format(d)+'_uz00'+"{:04d}".format(j)+'.dw', skiprows = 8, dtype = 'int', encoding='latin-1')
            vz = np.loadtxt(path+'/result/'+"0000" +"{:02d}".format(i) +"_R010_H0"+ str(h) +".0_S00"+str(s)+".00_D"+"{:003d}".format(d)+'_vz00'+"{:04d}".format(j)+'.dw', skiprows = 8, dtype = 'int', encoding='latin-1')
            stacked_uv = np.dstack((uz,vz))
            if j == 3600:
                Y.append(stacked_uv)
            else :
                time_images.append(stacked_uv)
        time_images = np.array(time_images)
        X2.append(time_images)


"\nearly_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)\nhistory = model.fit(x=[X1, X2, X3], y=Y, validation_split=0.2, epochs=50, callbacks=[early_stop])\n"

In [59]:
X1 = np.array(X1)
X2 = np.array(X2)
X3 = np.array(X3)
Y = np.array(Y)

(72, 1200, 1500, 2)


(72, 9, 1200, 1500, 2)


In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
history = model.fit(x=[X1, X2, X3], y=Y, validation_split=0.2, epochs=50, callbacks=[early_stop])